# Data preparation for Ghana adaptation example



In [ ]:
#data folder link for MAC
data_folder = "/Users/NAME/Desktop/Ghana_tutorial"
#data folder link for Windows users
data_folder = "C:\\Users\\NAME\\Desktop\\Ghana_tutorial"

In [ ]:
#Load libraries

In [ ]:
import os 
import sys 
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt 
import subprocess 
import csv 
import json 
import math 
import os 

from collections import namedtuple, OrderedDict 

import cartopy.crs as ccrs 
import cartopy.io.shapereader as shpreader 
from collections import namedtuple, OrderedDict 
import fiona 
import geopandas as gpd 
import matplotlib.patches as mpatches 
import matplotlib.pyplot as plt 
import numpy as np 

  
from boltons.iterutils import pairwise 

from geopy.distance import vincenty 

from osgeo import gdal 

from osgeo import gdal 
import shapely.geometry 
import shapely.ops 
from osgeo import gdal 
from scipy.spatial import Voronoi 
from shapely.geometry import Polygon, shape 

## Activity 1: Extract infrastructure data
### Step 1) ...

In [ ]:
#On your desktop, create a folder called Ghana_tutorial

#### Step 2) ...

In [ ]:
#type in the path towards your desktop, by changing NAME with the name as shown on your computer

In [ ]:
basepath = '...'#data folder link for MAC
data_folder = "/Users/lena/Desktop/Ghana_tutorial/"
data_folder = "/Users/NAME/Desktop/Ghana_tutorial/"

#data folder link for Windows users
#data_folder = "XX"
#Sarah to input how it looks for Windowsdata_folder = "C:\\Users\\NAME\\Desktop\\Ghana_tutorial"


#### Step 3) ...

In [ ]:
#Load libraries

import sys 
import geopandas as gpd 
import pandas as pd 
%matplotlib notebook 
import matplotlib.pyplot as plt 
import subprocess 
import csv 
import json 
import math 
import os 
from collections import namedtuple, OrderedDict 
import fiona 
import geopandas as gpd 
import matplotlib.patches as mpatches 
import matplotlib.pyplot as plt 
import numpy as np 
from osgeo import gdal 
import shapely.geometry 
import shapely.ops 
from osgeo import gdal 
from scipy.spatial import Voronoi 
from shapely.geometry import Polygon, shape 

#### Step 4) and 5) Download data and save data

In [ ]:
#Download the dataset in http://download.geofabrik.de/africa/ghana.html, extract the zip folder and save the extracted folder within your new folder 'Ghana_tutorial'

#### Step 6) Load the road dataset you've just downloaded

In [ ]:
GHA_OSM_roads = gpd.read_file(os.path.join(data_folder,'ghana-latest-free.shp', 'gis_osm_roads_free_1.shp'))

#### Step 7) To take a look at the data and the attribute table fill in and run the next two cells

In [ ]:
GHA_OSM_roads.head()

#### Step 8) Next we want to make a couple of changes to the data

In [ ]:
GHA_OSM_roads = GHA_OSM_roads[['fclass','name','oneway','bridge','tunnel','geometry']]
GHA_OSM_roads = GHA_OSM_roads.reset_index()
GHA_OSM_roads = GHA_OSM_roads.rename(columns = 
                                     {'index':'ID','fclass':'TYPE','name':'NAME'})

In [ ]:
GHA_OSM_roads.head()

#### Step 9) Save the pre-processed dataset

In [ ]:
GHA_OSM_roads.to_file(os.path.join(
data_folder,'GHA_OSM_roads.gpkg'),layer='OSM-roads', driver="GPKG")

## Activity 2: Extract and polygonise hazard data

#### Step 1) In the first step you've downloaded the flood data as tiff files from the Aqueduct database on GitHub

#### Step 2) In Step 2 you've created a new folder named *flood_layer* in your *data_folder* and saved your flood data 

#### Step 3) Now we want to convert the flood maps from *tiff files (raster data)* into *shape files (vector data)*

In [ ]:
#As a second step, we will download the hazard flood layers, provided by Aquaduct in the form of tiff files. These need to be converted (poligonised) into shapefiles in order to perform the intersection between hazards and infrastructure assets. To extract and poligonise the flood hazards, follow these steps:  

#### 1) Download a few tif files from Aqueduct:  https://github.com/nismod/aqueduct/blob/main/tiffs.txt 

In [ ]:
#Save all downloaded tif files in a new folder titled ‘flood_layer’ under your data_folder (home directory, see Activity 1, Step 1  (for example: Desktop/Ghana_tutorial)

#### Step 2) Run the below code to polygonise the tif files

In [ ]:
#Running the below code might take a while 

In [ ]:
for root, dirs, files in os.walk(data_folder, 'flood_layer'): 
    print(root) 
    print(dirs) 

for file in files: 
    if file.endswith("tif"): 
        print("Found tif file",file[:-4]) 
        output_file = file[:-4] + ".shp" 
        print(output_file) 
        output_file_path = os.path.join(data_folder, output_file) 
        input_file_path = os.path.join(data_folder, file) 
        output_file_path = os.path.join(data_folder, 'flood_layer',output_file) 
        input_file_path = os.path.join(data_folder, 'flood_layer', file) 
        print(output_file_path) 
        subprocess.run(["gdal_polygonize.py", input_file_path,'-q','-f', 'ESRI SHAPEFILE',output_file_path]) 

## Activity 3: Intersect hazard ...

#let us now intersect the hazard and the roads, starting with one hazard initially so we save time

#### Step 1) Specify your input and output path as well as the name of the intersection

In [ ]:
flood_gpd = gpd.read_file(os.path.join(data_folder,'flood_layer','inuncoast_rcp4p5_nosub_2050_rp0050_0.shp'))
output_path = os.path.join(data_folder, 'results')           
                                   

In [ ]:
## We now need to change and align the projection of the flood geodataframe we are using 

In [ ]:
#flood_gpd = flood_gpd.to_crs("EPSG:32630")

In [ ]:
def line_length(line, ellipsoid='WGS-84'): 

    """Length of a line in meters, given in geographic coordinates 

    Adapted from https://gis.stackexchange.com/questions/4022/looking-for-a-pythonic-way-to-calculate-the-length-of-a-wkt-linestring#answer-115285 

    Args: 

        line: a shapely LineString object with WGS-84 coordinates. 

        ellipsoid: string name of an ellipsoid that `geopy` understands (see http://geopy.readthedocs.io/en/latest/#module-geopy.distance). 

    Returns: 

        Length of line in kilometers. 

    """ 
    if line.geometryType() == 'MultiLineString': 
        return sum(line_length(segment) for segment in line) 
    return sum( 
        vincenty(tuple(reversed(a)), tuple(reversed(b)), ellipsoid=ellipsoid).kilometers 
        for a, b in pairwise(line.coords) 
    ) 

def intersections(road_gpd,flood_gpd,output_path): 
    data=[] 
    poly_sindex = flood_gpd.sindex 
    for l,lines in road_gpd.iterrows(): 
        if lines.geometry: 
            intersected_polys = flood_gpd.iloc[list(poly_sindex.intersection(lines.geometry.bounds))] 
            for p,poly in intersected_polys.iterrows(): 
                if (lines['geometry'].intersects(poly['geometry']) is True) and (poly.geometry.is_valid is True): 
                    geom = lines['geometry'].intersection(poly['geometry']) 
                    data.append({'gid':lines['gid'],'flood_depth':poly['DN'],'flood_length':geom.length,'geometry':geom}) 

    if data: 
        intersections_data = gpd.GeoDataFrame(data, columns=['gid','flood_depth', 'flood_length', 'geometry'], crs='epsg:4326') 
        intersections_data.to_file(os.path.join(output_path,'flood_results.shp')) 
        return intersections_data 

def convert(threshold, infile, outfile): 

    """Convert GeoTiff raster file to Shapefile with geometries based on raster threshold less that 999 
    Parameters 
        - threshold - Float value of lower bound of GeoTiff threshold value to be selected 
        - infile - String name of input GeoTff file path 
        - tmpfile_1 - Stirng name of tmp file 1 
        - outfile - Stirng name of output shapefile 

    Outputs 
        Shapefile with Polygon geometries of rasters based on raster values above a threshold 

    """ 
    subprocess.run([ 
        "gdal_polygonize.py", 
        tmpfile_1, 
        '-q', 
        '-f', 'ESRI Shapefile', 
        outfile 
    ]) 

#### Step 2) Running the intersection

In [ ]:
#We now run the intersection
intersections(GHA_OSM_roads,flood_gpd,output_path) 

## Activity 4: QGIS

Activity 4 is done in QGIS ...